In [1]:
import sys
#sys.path.append('C:/Pradap/Research/Python-Packages/enrique/')
sys.path.append('/scratch/pradap/python-work/enrique/')

In [2]:
import jpype

In [3]:
jpype.getDefaultJVMPath()

'/scratch/pradap/local/share/jdk1.7.0_25/jre/lib/amd64/server/libjvm.so'

In [4]:
import magellan as mg

SyntaxError: invalid syntax (addfeatures.py, line 151)

In [ ]:
mg._m_global_tokenizers

In [ ]:
t = mg.get_single_arg_tokenizers()

In [ ]:
s = 'hello world'

In [ ]:
#mg.init_jvm('C:\\Program Files\\Java\\jre7\\bin\\server\\jvm.dll')
mg.init_jvm('/scratch/pradap/local/share/jdk1.7.0_25/jre/lib/amd64/server/libjvm.so')

In [ ]:
l = t['qgm_3']('hello world')

In [ ]:
l

In [ ]:
s = mg.get_sim_funs()

In [ ]:
s['jaccard'](l, l)

In [ ]:
s = mg.get_sim_funs()
t = mg.get_single_arg_tokenizers()

In [ ]:
st = 'jaccard(qgm_2(ltuple["name"]), qgm_2(rtuple["name"]))'

In [ ]:
d['function'](A.ix[1], B.ix[2])

In [ ]:
d = mg.get_feature_fn(st, s, t)

In [ ]:
f = d['function']

In [ ]:
#i/o
A = mg.read_csv('../magellan/datasets/table_A.csv', key = 'ID')
B = mg.read_csv('../magellan/datasets/table_B.csv', key = 'ID')

In [ ]:
d1 = {}
d1['f'] = d['function']

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame([d, d])

In [ ]:
df.loc[2] = d #append(d, ignore_index=True)

In [ ]:
df.loc[3] = d

In [ ]:
d